In [24]:
# === SETUP ===
INPUT_FILE = "/content/drive/MyDrive/KlikBERT/labeled_dataset_clickbait.csv"
OUTPUT_FILE = "/content/drive/MyDrive/KlikBERT/labeled_dataset_complete.csv" # Nama file output baru

CHUNK_SIZE = 500      # Seberapa besar batch dibaca dari disk
N_WORKERS  = 8        # Jumlah proses paralel (Colab bisa 2-4 CPU)

# ==============================================================================
# JANGAN UBAH BAGIAN DI BAWAH INI
# ==============================================================================

from google import genai
from google.genai import types
import os
import pandas as pd
import json
from tqdm.auto import tqdm
from multiprocessing import Pool, cpu_count
import time
from random import uniform
from google.colab import userdata
from google.colab import drive


drive.mount("/content/drive")

# === SET YOUR API KEY ENV VAR ===
try:
    os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")
    api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key:
        raise ValueError("API Key not found in Colab Secrets")
except Exception as e:
    print(f"Error saat mengambil API Key. Pastikan Anda sudah menyimpannya di Colab Secrets dengan nama 'GOOGLE_API_KEY'.\nError: {e}")


# === GENERATE FUNCTION ===
# Fungsi ini tidak perlu diubah. Ia menerima (id, judul, isi) dan mengembalikan (id, judul, isi, kategori)
def call_gemini(args):
    row_id, judul, isi = args
    valid_categories = ['Politik', 'Kesehatan', 'Lingkungan', 'Teknologi', 'Bisnis', 'Entertainment', 'Lifestyle', 'Sport', 'Kriminal']
    try:
        client = genai.Client(api_key=api_key)
        model = "gemini-2.5-flash-lite-preview-06-17"

        input_text = f"Judul: {judul}\nIsi: {isi}\nKategori:"
        contents = [types.Content(role="user", parts=[types.Part.from_text(text=input_text)])]

        generate_content_config = types.GenerateContentConfig(
            temperature=0,
            thinking_config=types.ThinkingConfig(thinking_budget=0),
            response_mime_type="application/json",
            response_schema=genai.types.Schema(
                type=genai.types.Type.OBJECT,
                required=["kategori"],
                properties={"kategori": genai.types.Schema(type=genai.types.Type.STRING)},
            ),
            system_instruction=[
                types.Part.from_text(text="""Anda adalah seorang ahli editor berita yang sangat teliti dan patuh pada aturan.

Tugas utama Anda adalah mengklasifikasikan berita yang saya berikan ke dalam **satu** kategori saja dari 9 pilihan di bawah ini. Jawaban Anda **wajib** hanya berupa **satu kata** nama kategori tersebut dan tidak boleh ada tambahan teks apa pun.

**DAFTAR KATEGORI WAJIB:**
* Politik
* Kesehatan
* Lingkungan
* Teknologi
* Bisnis
* Entertainment
* Lifestyle
* Sport
* Kriminal

---
**PANDUAN WAJIB: Cara Memilih Kategori untuk Topik Khusus**

Anda harus mengikuti panduan ini jika menemukan berita dengan topik yang tidak ada di daftar utama. Pilih kategori yang paling sesuai berdasarkan sudut pandang berita:

* **Untuk topik tentang PENDIDIKAN:**
    * Jika membahas kebijakan pemerintah, anggaran, atau pernyataan menteri, pilih **Politik**.
    * Jika membahas aplikasi belajar online, platform, atau penggunaan alat teknologi di sekolah, pilih **Teknologi**.
    * Jika membahas kinerja keuangan perusahaan pendidikan atau bimbel, pilih **Bisnis**.
    * Jika membahas tips belajar, aktivitas siswa, atau tren gaya hidup di kampus, pilih **Lifestyle**.

* **Untuk topik tentang HUKUM:**
    * Jika membahas tindak pidana atau investigasi kejahatan, pilih **Kriminal**.
    * Jika membahas proses pembuatan undang-undang di DPR atau putusan MK, pilih **Politik**.
    * Jika membahas sengketa antar perusahaan atau hukum dagang, pilih **Bisnis**.

* **Untuk topik tentang SAINS:**
    * Jika membahas penemuan medis atau biologi manusia, pilih **Kesehatan**.
    * Jika membahas iklim, geologi, atau spesies baru, pilih **Lingkungan**.
    * Jika membahas penemuan fisika, astronomi, atau yang mendasari inovasi, pilih **Teknologi**.

* **Untuk topik tentang SOSIAL atau BENCANA ALAM:**
    * Jika membahas respons pemerintah atau pengerahan bantuan, pilih **Politik**.
    * Jika membahas penyebab ilmiah dari bencana tersebut, pilih **Lingkungan**.
    * Jika membahas kisah inspiratif para korban atau gaya hidup komunitas, pilih **Lifestyle**.

---
**CONTOH CARA ANDA MENJAWAB:**

**Contoh 1:**
* **Judul Berita:** "Mendikbud Umumkan Kurikulum Prototipe, Gantikan Kurikulum 2013"
* **Isi Berita:** "Menteri Pendidikan Nadiem Makarim resmi memperkenalkan Kurikulum Prototipe sebagai opsi pemulihan pembelajaran. Kebijakan ini akan dievaluasi secara berkala oleh kementerian..."
* **Jawaban Anda:** Politik

**Contoh 2:**
* **Judul Berita:** "Kecelakaan Maut di Tol Cipali Libatkan Tiga Kendaraan"
* **Isi Berita:** "Terjadi kecelakaan beruntun di ruas tol Cipali KM 117 yang menewaskan dua orang. Pihak kepolisian masih menyelidiki penyebab pasti dari tabrakan tersebut..."
* **Jawaban Anda:** Kriminal
---

Sekarang, klasifikasikan berita di bawah ini.
Ingat, jawaban Anda **hanya satu kata**.

**Judul:**
[tempelkan judul berita di sini]

**Isi Berita:**
[tempelkan isi berita di sini]

**Kategori:**""")
            ],
        )

        full_response = ""
        for chunk in client.models.generate_content_stream(model=model, contents=contents, config=generate_content_config):
            if chunk.text:
                full_response += chunk.text

        result = json.loads(full_response)
        kategori = result.get("kategori")

        # Fallback mechanism: Check if the returned category is in the valid list
        if kategori not in valid_categories:
            print(f"⚠️ Invalid category returned for id={row_id}: {kategori}. Setting to None.")
            kategori = None

        return (row_id, judul, isi, kategori)
    except Exception as e:
        print(f"❌ ERROR (id={row_id}): {e}")
        return (row_id, judul, isi, None)


# === MAIN PIPELINE (LOGIKA DIGUBAH) ===
def run_batch_labeling():
    # 1. Cek file output untuk melanjutkan proses
    output_exists = os.path.exists(OUTPUT_FILE)
    if output_exists:
        print(f"✅ Membaca output lama dari '{OUTPUT_FILE}' untuk melanjutkan...")
        try:
            # Baca hanya kolom id untuk mengecek baris mana yang sudah selesai
            done_ids = set(pd.read_csv(OUTPUT_FILE, usecols=['id'])['id'].tolist())
            print(f"✅ Sudah ada {len(done_ids)} baris yang diproses.")
        except Exception as e:
            print(f"⚠️ Gagal membaca file output lama: {e}. Menganggap file kosong.")
            done_ids = set()
    else:
        done_ids = set()
        print("✅ File output belum ada. Memulai dari awal.")

    # 2. Buka file input untuk dibaca per chunk
    reader = pd.read_csv(INPUT_FILE, chunksize=CHUNK_SIZE)
    for chunk in reader:
        # Saring baris yang sudah ada di file output
        original_chunk_to_process = chunk[~chunk["id"].isin(done_ids)].copy()
        if original_chunk_to_process.empty:
            continue

        print(f"\n✨ Memproses batch baru: {len(original_chunk_to_process)} baris")
        # Siapkan data untuk multiprocessing dari chunk yang akan diproses
        records = [(row.id, row.judul, row.isi) for row in original_chunk_to_process.itertuples(index=False)]

        # 3. Jalankan LLM secara paralel
        with Pool(processes=N_WORKERS) as pool:
            results = list(tqdm(pool.imap(call_gemini, records), total=len(records)))

        # 4. Buat DataFrame dari hasil LLM
        # Hanya butuh id dan kategori untuk digabungkan
        results_df = pd.DataFrame(results, columns=["id", "judul_temp", "isi_temp", "kategori"])

        # 5. Gabungkan (merge) DataFrame asli dengan hasil LLM berdasarkan 'id'
        # Ini akan mempertahankan semua kolom asli dan menambahkan kolom 'kategori'
        chunk_final = pd.merge(original_chunk_to_process, results_df[['id', 'kategori']], on='id', how='left')

        # 6. Salin kolom 'kategori' dari merged_df ke original_chunk_to_process (jika kategori ada)
        # chunk_final['kategori'] = chunk_final['kategori_y'].combine_first(chunk_final['kategori_x'])


        # 7. Simpan chunk yang sudah digabung ke file output
        # Tulis header hanya jika file belum ada
        header_needed = not output_exists
        chunk_final.to_csv(OUTPUT_FILE, mode='a', header=header_needed, index=False)
        output_exists = True # Setelah penulisan pertama, pastikan header tidak ditulis lagi

        print(f"✅ Batch selesai disimpan. Total baris di output: {len(done_ids) + len(chunk_final)}")
        done_ids.update(chunk_final["id"]) # Update set ID yang sudah selesai
        time.sleep(uniform(1, 3))


# === RUN ===
if __name__ == "__main__":
    run_batch_labeling()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Membaca output lama dari '/content/drive/MyDrive/KlikBERT/labeled_dataset_complete.csv' untuk melanjutkan...
✅ Sudah ada 45527 baris yang diproses.

✨ Memproses batch baru: 1 baris


  0%|          | 0/1 [00:00<?, ?it/s]

✅ Batch selesai disimpan. Total baris di output: 45528

✨ Memproses batch baru: 2 baris


  0%|          | 0/2 [00:00<?, ?it/s]

⚠️ Invalid category returned for id=4255: Sosial. Setting to None.
⚠️ Invalid category returned for id=4392: Sosial. Setting to None.
✅ Batch selesai disimpan. Total baris di output: 45530

✨ Memproses batch baru: 1 baris


...

⚠️ Invalid category returned for id=40606: Sosial. Setting to None.
✅ Batch selesai disimpan. Total baris di output: 45573

✨ Memproses batch baru: 1 baris


  0%|          | 0/1 [00:00<?, ?it/s]

⚠️ Invalid category returned for id=44694: Kecelakaan. Setting to None.
✅ Batch selesai disimpan. Total baris di output: 45574

✨ Memproses batch baru: 1 baris


  0%|          | 0/1 [00:00<?, ?it/s]

⚠️ Invalid category returned for id=45120: Hukum. Setting to None.
✅ Batch selesai disimpan. Total baris di output: 45575


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/KlikBERT/to_retry_ok.csv")

valid_categories = ['Politik', 'Kesehatan', 'Lingkungan', 'Teknologi', 'Bisnis', 'Entertainment', 'Lifestyle', 'Sport', 'Kriminal']

# Filter rows where 'kategori' is null OR 'kategori' is not in the valid_categories list
gagal = df[df['kategori'].isnull() | ~df['kategori'].isin(valid_categories)].copy()

print(f"Jumlah baris gagal: {len(gagal)}")

# Display the unique invalid categories found
invalid_categories_found = gagal[~gagal['kategori'].isin(valid_categories)]
if not invalid_categories_found.empty:
    print("Ditemukan kategori yang tidak termasuk dalam daftar:")
    display(invalid_categories_found[['id', 'kategori']].drop_duplicates(subset=['kategori']))
else:
    print("Semua kategori yang tidak null termasuk dalam daftar yang valid.")


gagal.to_csv("/content/drive/MyDrive/KlikBERT/to_retry.csv", index=False)

Jumlah baris gagal: 8
Ditemukan kategori yang tidak termasuk dalam daftar:


,id,kategori
6,4255,Sosial
7,5718,Pendidikan
84,8949,NaN
